# Issue 46: add functionality to choose minimizing or maximizing for each metric in a dataset

[See issue progress on GitHub here.](https://github.com/NREL/tyche/issues/46)

In [1]:
import os
import sys

sys.path.insert(1, os.path.abspath("../../src"))

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

# Sustainable Aviation Fuel

In [2]:
saf_loc = "../../tutorial/data/saf"

saf_designs = ty.Designs(saf_loc)
saf_investments = ty.Investments(saf_loc)
saf_designs.compile()
tranche_results = saf_investments.evaluate_tranches(saf_designs,
                                                    sample_count=1000)

techs = tranche_results.summary.index.get_level_values('Technology').unique()

Evaluating HEFA Camelina
Evaluating HEFA Castor
Evaluating HEFA Jatropha
Evaluating HEFA Pennycress
Evaluating HEFA Yellow Grease


## Using the `opt_slsqp` (formerly `min_slsqp`) method

Evaluate the maximum metric values using the `optimum_metrics` method (formerly the `max_metrics` method). After refactoring, the method can take either a `Dict` of optimization senses, one per metric, or a `str` specifying one optimization sense to be used across all metrics. If no sense is provided, then the optimization sense used to instantiate the `EpsilonConstraintOptimizer` class object is used for all metrics.

First demonstrate the use of an optimization sense dictionary, with one sense per metric.

In [3]:
sense_dict = {'Reduction in MJSP' : 'max',
              'Reduction in Jet GHG': 'max',
              'MJSP': 'min',
              'Jet GHG': 'min'}

opt_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min'
    )
    opt_metric_values[i] = optimizer.optimum_metrics(
        metric_sense = sense_dict,
        total_amount = 5000000
    )

opt_metric_values

{'HEFA Camelina': Jet GHG                  0.086806
 MJSP                    26.558133
 Reduction in Jet GHG     0.054194
 Reduction in MJSP        2.457708
 Name: Value, dtype: float64,
 'HEFA Castor': Jet GHG                 1.590000e-01
 MJSP                    2.864583e+01
 Reduction in Jet GHG   -9.912778e-18
 Reduction in MJSP       2.339883e+00
 Name: Value, dtype: float64,
 'HEFA Jatropha': Jet GHG                 1.650000e-01
 MJSP                    1.389508e+01
 Reduction in Jet GHG   -1.859212e-17
 Reduction in MJSP       1.131671e+00
 Name: Value, dtype: float64,
 'HEFA Pennycress': Jet GHG                 9.810000e-02
 MJSP                    2.349886e+01
 Reduction in Jet GHG   -5.859814e-18
 Reduction in MJSP       1.354039e+00
 Name: Value, dtype: float64,
 'HEFA Yellow Grease': Jet GHG                  0.041724
 MJSP                    14.452646
 Reduction in Jet GHG     0.009876
 Reduction in MJSP        1.409284
 Name: Value, dtype: float64}

Next, provide a sense (max) as a string to `optimum_metrics` that overrides the sense provided to the `EpsilonConstraintOptimizer` object.

In [5]:
max_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min',
    )
    max_metric_values[i] = optimizer.optimum_metrics(
        metric_sense = 'max',
        total_amount = 5000000
    )

max_metric_values

{'HEFA Camelina': Jet GHG                  0.141000
 MJSP                    29.015842
 Reduction in Jet GHG     0.054194
 Reduction in MJSP        2.457708
 Name: Value, dtype: float64,
 'HEFA Castor': Jet GHG                 2.303947e-01
 MJSP                    3.098572e+01
 Reduction in Jet GHG   -9.912778e-18
 Reduction in MJSP       2.339883e+00
 Name: Value, dtype: float64,
 'HEFA Jatropha': Jet GHG                 3.000780e-01
 MJSP                    1.502675e+01
 Reduction in Jet GHG   -1.859212e-17
 Reduction in MJSP       1.131671e+00
 Name: Value, dtype: float64,
 'HEFA Pennycress': Jet GHG                 1.304412e-01
 MJSP                    2.485290e+01
 Reduction in Jet GHG   -5.859814e-18
 Reduction in MJSP       1.354039e+00
 Name: Value, dtype: float64,
 'HEFA Yellow Grease': Jet GHG                  0.051600
 MJSP                    15.861930
 Reduction in Jet GHG     0.009876
 Reduction in MJSP        1.409284
 Name: Value, dtype: float64}

Optimize for minimum jet selling price (MJSP) by minimizing MJSP directly.

In [6]:
min_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_results.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            0.000000e+00
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090807
  MJSP                    26.558133
  Reduction in Jet GHG     0.050193
  Reduction in MJSP        2.457708
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            5.500000e+05
  Fatty Acid Composition        1.750000e+06
  Overall Process Efficiency    2.700000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.178170
  MJSP                    28.645826
  Reduction in Jet GHG    -0.019170
  Reduction in MJSP        2.339884
  Name: Value, dtype: float64},
 {'Tec

Now optimize for MJSP by maximizing the reduction in MSJP from the state of technology value. This will determine if the same solutions are found when minimizing MJSP versus maximizing the reduction in MJSP.

In [10]:
max_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Reduction in MJSP",
        sense = 'max',
        total_amount = 5000000        
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_results.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.236428e-04
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090807
  MJSP                    26.558133
  Reduction in Jet GHG     0.050193
  Reduction in MJSP        2.457708
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            5.500015e+05
  Fatty Acid Composition        1.749999e+06
  Overall Process Efficiency    2.700000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.178170
  MJSP                    28.645827
  Reduction in Jet GHG    -0.019170
  Reduction in MJSP        2.339883
  Name: Value, dtype: float64},
 {'Tec

The MJSP values match when MJSP is minimized or when the reduction in MJSP is maximized.

Now perform the same test on the GHG metrics. First, minimize jet GHG.

In [11]:
min_ghg = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Jet GHG",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_ghg.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_ghg

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            5.974842e+05
  Fatty Acid Composition        2.599999e+06
  Overall Process Efficiency    1.802517e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.086806
  MJSP                    26.889541
  Reduction in Jet GHG     0.054194
  Reduction in MJSP        2.126300
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            2.341572e-11
  Fatty Acid Composition        5.065393e-10
  Overall Process Efficiency    3.708937e-10
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                 1.590000e-01
  MJSP                    3.098572e+01
  Reduction in Jet GHG   -1.385508e-17
  Reduction in MJSP      -1.498411e-05
  Name: Value, dtype: floa

Now, maximize the reduction in jet GHG.

In [13]:
max_red_ghg = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Reduction in Jet GHG",
        sense = 'max',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_red_ghg.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_red_ghg

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            5.974844e+05
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    1.802516e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.086806
  MJSP                    26.889541
  Reduction in Jet GHG     0.054194
  Reduction in MJSP        2.126300
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.447113e-10
  Fatty Acid Composition        1.214306e-10
  Overall Process Efficiency    3.708937e-10
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                 1.590000e-01
  MJSP                    3.098572e+01
  Reduction in Jet GHG   -9.912778e-18
  Reduction in MJSP      -1.498411e-05
  Name: Value, dtype: floa

These optimal values match as well when jet GHG is minimized versus when the reduction in jet GHG is maximized.

## Using the `opt_milp` (formerly `pwlinear_milp`) method

In [3]:
min_mjsp = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_milp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_mjsp.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_mjsp

HEFA Camelina - OptimizationStatus.OPTIMAL
HEFA Castor - OptimizationStatus.OPTIMAL
HEFA Jatropha - OptimizationStatus.OPTIMAL
HEFA Pennycress - OptimizationStatus.OPTIMAL
HEFA Yellow Grease - OptimizationStatus.OPTIMAL


[{'Technology': 'HEFA Camelina',
  'Optimizer status': <OptimizationStatus.OPTIMAL: 0>,
  'Investment amounts': Category
  Co-Product Revenue                  0.0
  Fatty Acid Composition        2300000.0
  Overall Process Efficiency    2700000.0
  Name: Amount, dtype: float64,
  'Optimum metrics': Jet GHG                  0.000000
  MJSP                     0.000000
  Reduction in Jet GHG     0.000000
  Reduction in MJSP       15.348357
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': <OptimizationStatus.OPTIMAL: 0>,
  'Investment amounts': Category
  Co-Product Revenue             550000.0
  Fatty Acid Composition        1750000.0
  Overall Process Efficiency    2700000.0
  Name: Amount, dtype: float64,
  'Optimum metrics': Jet GHG                 0.0
  MJSP                    0.0
  Reduction in Jet GHG    0.0
  Reduction in MJSP       0.0
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Jatropha',
  'Optimizer status': <OptimizationStatus.OPT

In [4]:
max_red_mjsp = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_milp(
        "Reduction in MJSP",
        sense = 'max',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_red_mjsp.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_red_mjsp

HEFA Camelina - OptimizationStatus.OPTIMAL
HEFA Castor - OptimizationStatus.OPTIMAL
HEFA Jatropha - OptimizationStatus.OPTIMAL
HEFA Pennycress - OptimizationStatus.OPTIMAL
HEFA Yellow Grease - OptimizationStatus.OPTIMAL


[{'Technology': 'HEFA Camelina',
  'Optimizer status': <OptimizationStatus.OPTIMAL: 0>,
  'Investment amounts': Category
  Co-Product Revenue                  0.0
  Fatty Acid Composition        2300000.0
  Overall Process Efficiency    2700000.0
  Name: Amount, dtype: float64,
  'Optimum metrics': Jet GHG                  0.000000
  MJSP                     0.000000
  Reduction in Jet GHG     0.000000
  Reduction in MJSP       15.348357
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': <OptimizationStatus.OPTIMAL: 0>,
  'Investment amounts': Category
  Co-Product Revenue             550000.0
  Fatty Acid Composition        1750000.0
  Overall Process Efficiency    2700000.0
  Name: Amount, dtype: float64,
  'Optimum metrics': Jet GHG                 0.0
  MJSP                    0.0
  Reduction in Jet GHG    0.0
  Reduction in MJSP       0.0
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Jatropha',
  'Optimizer status': <OptimizationStatus.OPT

## Epsilon constraint formulation using optimization sense

All epsilon constraints are formulated as value - limit >= 0, hence LHS >= RHS. There is already a sense parameter in the method used to construct the "value" or LHS of the constraint, that applies a negative to the value when the sense is "maximum" and does not apply a negative when the sense is "minimum". (Actual parameter values are 'min' and 'max' respectively.) 

To construct a lower-bound epsilon constraint, no negative is applied to the "value" and the constraint is formulated as value - limit >= 0 --> value >= limit. This happens when the `eps_sense` parameter on the optimization method (`opt_slsqp` or `opt_milp`) is set to 'min'.

To construct an upper-bound epsilon constraint, the constraint is formulated as -1 * value - limit >= 0 --> value <= limit. This happens when `eps_sense` is set to 'max'.

`eps_sense` defaults to 'min', thus by default epsilon constraints are constructed as lower bounds on the metrics.

Demonstrate this functionality by minimizing MJSP (sense = 'min') while constraining the reduction in jet GHG emissions to be greater than zero (eps_sense = 'min').

In [3]:
i = techs[0]

tranche_tech_results = tranche_results.summary[
    tranche_results.summary.index.get_level_values('Technology') == i
]
evaluator = ty.Evaluator(
    saf_investments.tranches, tranche_tech_results
)
optimizer = ty.EpsilonConstraintOptimizer(
    evaluator
)
w = optimizer.opt_slsqp(
    "MJSP",
    sense = 'min',
    total_amount = 5000000,
    eps_metric = pd.Series([0.0], name="Value", index = ["Reduction in Jet GHG"])
)

In [6]:
eps_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000,
        eps_metric = pd.Series([0.0], name="Value", index = ["Reduction in Jet GHG"]),
        eps_sense = pd.Series(['min'], name="Value", index = ["Reduction in Jet GHG"])
    )
    print(f"{i} - {w[1]}")
    eps_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    eps_results.append(
        eps_dict
    )
    del tranche_tech_results, evaluator, optimizer, eps_dict, w

eps_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            6.315577e-05
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090879
  MJSP                    26.558054
  Reduction in Jet GHG     0.050121
  Reduction in MJSP        2.457787
  Name: Value, dtype: float64},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            0.000000e+00
  Fatty Acid Composition        7.802732e-10
  Overall Process Efficiency    0.000000e+00
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                 1.590000e-01
  MJSP                    3.098572e+01
  Reduction in Jet GHG   -1.123315e-17
  Reduction in MJSP      -1.498411e-05
  Name: Value, dtype: floa

The reduction in jet GHG values in the optimal solutions are within the tolerance value.